<a href="https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_multilenguaje_cuaderno_de_entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="ffc800"> **Cuaderno de entrenamiento de [Piper.](https://github.com/rhasspy/piper)**
## ![Piper logo](https://contribute.rhasspy.org/img/logo.png)

---

- Cuaderno creado por [rmcpantoja](http://github.com/rmcpantoja)
- Colaborador y traductor: [Xx_Nessu_xX](http://github.com/Xx_Nessu_xX)

---

# Notas:

- <font color="orange">**Las cosas en naranja significa que son importantes.**

# Créditos:

* [Feanix-Fyre fork](https://github.com/Feanix-Fyre/piper) con algunas mejoras.
* [Tacotron2 NVIDIA training notebook](https://github.com/justinjohn0306/FakeYou-Tacotron2-Notebook) - Fragmento de duración del conjunto de datos.
* [🐸TTS](https://github.com/coqui-ai/TTS) - Demostración del remuestreador y del formador XTTS.

# <font color="ffc800">🔧 ***Primeros pasos.*** 🔧

In [ ]:
#@markdown ## <font color="ffc800"> **Google Colab Anti-Disconnect.** 🔌
#@markdown ---
#@markdown #### Evita la desconexión automática. Aún así, se desconectará después de <font color="orange">**6 a 12 horas**</font>.

import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''
display(IPython.display.Javascript(js_code))

In [ ]:
#@markdown ## <font color="ffc800"> **Comprueba la GPU.** 👁️
#@markdown ---
#@markdown #### Una GPU de mayor capacidad puede aumentar la velocidad de entrenamiento. Por defecto, tendrás una <font color="orange">**Tesla T4**</font>.
!nvidia-smi

In [ ]:
#@markdown # <font color="ffc800"> **Monta tu Google Drive.** 📂
#@markdown ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@markdown # <font color="ffc800"> **Instalar software.** 📦
#@markdown ---

#@markdown ####En esta celda se instalará el sintetizador y sus dependencias necesarias para ejecutar el entrenamiento. (Esto puede llevar un rato.)

# clone:
!git clone -q https://github.com/rmcpantoja/piper
%cd /content/piper/src/python
!wget -q "https://raw.githubusercontent.com/coqui-ai/TTS/dev/TTS/bin/resample.py"
!pip install -q -r requirements.txt
!pip install -q cython>=0.29.0 piper-phonemize==1.1.0 librosa>=0.9.2 numpy==1.24 onnxruntime>=1.11.0 pytorch-lightning==1.7.7 torch==1.13.0+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install -q torchtext==0.14.0 torchvision==0.14.0
# fixing recent compativility isswes:
!pip install -q torchaudio==0.13.0 torchmetrics==0.11.4
!bash build_monotonic_align.sh
# Useful vars:
use_whisper = True
print("\033[93mHecho.")

# <font color="ffc800"> 🤖 ***Entrenamiento.*** 🤖

In [ ]:
#@markdown # <font color="ffc800"> **1. Extraer dataset.** 📥
#@markdown ---
#@markdown ####Importante: los audios deben estar en formato <font color="orange">**wav, (16000 o 22050hz, 16-bits, mono), y, por comodidad, numerados.<br>Ejemplo:**

#@markdown * <font color="orange">**1.wav**</font>
#@markdown * <font color="orange">**2.wav**</font>
#@markdown * <font color="orange">**3.wav**</font>
#@markdown * <font color="orange">**.....**</font>

#@markdown ---
import os
import wave
import zipfile
import datetime

def get_dataset_duration(wav_path):
    totalduration = 0
    for file_name in [x for x in os.listdir(wav_path) if os.path.isfile(x)]:
        with wave.open(file_name, "rb") as wave_file:
            frames = wave_file.getnframes()
            rate = wave_file.getframerate()
            duration = frames / float(rate)
            totalduration += duration
    wav_count = len(os.listdir(wav_path))
    duration_str = str(datetime.timedelta(seconds=round(totalduration, 0)))
    return wav_count, duration_str

%cd /content
if not os.path.exists("/content/dataset"):
    os.makedirs("/content/dataset")
    os.makedirs("/content/dataset/wavs")
%cd /content/dataset
#@markdown ### Ruta del dataset para descomprimir:
zip_path = "/content/drive/MyDrive/wavs.zip" #@param {type:"string"}
zip_path = zip_path.strip()
if zip_path:
    if os.path.exists(zip_path):
        if zipfile.is_zipfile(zip_path):
            print("Descomprimiendo audios...")
            !unzip -q -j "{zip_path}" -d /content/dataset/wavs
        else:
            print("Copiando audios en su directorio...")
            fp = zip_path + "/."
            !cp -a "$fp" "/content/dataset/wavs"
    else:
        raise Exception("La ruta proporcionada para los wavs no es correcta. Por favor, introduzca una ruta válida.")
else:
    raise Exception("Debes proporcionar una ruta a los wavs.")
if os.path.exists("/content/dataset/wavs/wavs"):
    for file in os.listdir("/content/dataset/wavs/wavs"):
        !mv /content/dataset/wavs/wavs/"$file"  /content/dataset/wavs/"$file"
    !rm -r /content/dataset/wavs/*.txt
    !rm -r /content/dataset/wavs/*.csv
%cd /content/dataset/wavs
audio_count, dataset_dur = get_dataset_duration("/content/dataset/wavs")
print(f"Conjunto de datos abierto con {audio_count} wavs con una duración de {dataset_dur}.")
%cd ..
#@markdown ---

In [ ]:
#@markdown # <font color="ffc800"> **2. Cargar el archivo de transcripción.** 📝
#@markdown ---
#@markdown ####<font color="orange">**Importante: la transcripción significa escribir lo que dice el personaje en cada uno de los audios, y debe tener la siguiente estructura:**

#@markdown ##### <font color="orange">Para un conjunto de datos de un solo hablante:
#@markdown * wavs/1.wav|Esto dice el personaje en el audio 1.
#@markdown * wavs/2.wav|Este, el texto que dice el personaje en el audio 2.
#@markdown * ...

#@markdown ##### <font color="orange">Para un conjunto de datos de varios hablantes:

#@markdown * wavs/speaker1audio1.wav|speaker1|Esto es lo que dice el primer hablante.
#@markdown * wavs/speaker1audio2.wav|speaker1|Este es otro audio del primer hablante.
#@markdown * wavs/speaker2audio1.wav|speaker2|Esto es lo que dice el segundo hablante en el primer audio.
#@markdown * wavs/speaker2audio2.wav|speaker2|Este es otro audio del segundo hablante.
#@markdown * ...

#@markdown #### Y así sucesivamente. Además, la transcripción debe estar en formato <font color="orange">**.csv o también sirve en .txt (UTF-8 sin BOM)**
#@markdown ---
#@markdown ## <font color="orange">**![¡NUEVO!](https://cdn.discordapp.com/attachments/821410297335513139/1187144741637337088/nuevo.gif?ex=6595d1b2&is=65835cb2&hm=986bc4552e5bda39b67136798f8b906b5c170620e44bb30d63b431fe23c6a590&) Auto-transcripción con Whisper IA si no se proporciona la transcripción.**

#@markdown ####**Nota: Si no subes ningún archivo de transcripción, los wavs se transcribirán utilizando la herramienta Whisper cuando ejecutes el siguiente paso. Después, el bloc de notas continuará con el resto del preprocesamiento si no hay errores. Aunque la herramienta Whisper tiene buenos resultados de transcripción, en mi experiencia recomiendo transcribir manualmente y subirlo desde esta celda, ya que una buena voz TTS necesita ser optimizada para dar aún mejores resultados. Por ejemplo, al transcribir manualmente podrás observar cada detalle que hace el hablante (como puntuación, sonidos, etc.), y plasmarlos en la transcripción de acuerdo a las entonaciones del hablante.**
#@markdown ---
%cd /content/dataset
from google.colab import files
!rm /content/dataset/metadata.csv
listfn, length = files.upload().popitem()
if listfn != "metadata.csv":
  !mv "$listfn" metadata.csv
use_whisper = False
%cd ..

In [ ]:
#@markdown # <font color="ffc800"> **3. Preprocesar el dataset.** 🔄
#@markdown ---
import os
if use_whisper:
    import torch
    from faster_whisper import WhisperModel
    from tqdm import tqdm
    from google import colab

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Utilizar el dispositivo: {device}")

    def make_dataset(path, language):
        metadata = ""
        text = ""
        files = [f for f in os.listdir(path) if f.endswith(".wav")]
        assert len(files) > 0, "¡Tampoco has subido los wavs! Por favor, sube al menos un zip con los wavs en el paso 2."
        metadata_file = open(f"{path}/../metadata.csv", "w")
        whisper = WhisperModel("large-v3", device=device, compute_type="float16")
        for audio_file in tqdm(files):
            full_path = os.path.join(path, audio_file)
            segments, _ = whisper.transcribe(full_path, word_timestamps=False, language=language)
            for segment in segments:
                text += segment.text
            text = text.strip()
            text = text.replace('\n', ' ')
            metadata = f"{audio_file}|{text}\n"
            metadata_file.write(metadata)
            text = ""
        colab.files.download(f"{path}/../metadata.csv")
        del whisper
        return True
#@markdown ### En primer lugar, seleccione el idioma de su conjunto de datos. <br> (Está disponible para español los siguientes: Español castellano y Español lationamericano.)
language = "Español (Castellano)" #@param ["ألعَرَبِي", "Català", "čeština", "Dansk", "Deutsch", "Ελληνικά", "English (British)", "English (U.S.)", "Español (Castellano)", "Español (Latinoamericano)", "Suomi", "Français", "Magyar", "Icelandic", "Italiano", "ქართული", "қазақша", "Lëtzebuergesch", "नेपाली", "Nederlands", "Norsk", "Polski", "Português (Brasil)", "Português (Portugal)", "Română", "Русский", "Српски", "Svenska", "Kiswahili", "Türkçe", "украї́нська", "Tiếng Việt", "简体中文"]
#@markdown ---
# language definition:
languages = {
    "ألعَرَبِي": "ar",
    "Català": "ca",
    "čeština": "cs",
    "Dansk": "da",
    "Deutsch": "de",
    "Ελληνικά": "el",
    "English (British)": "en",
    "English (U.S.)": "en-us",
    "Español (Castellano)": "es",
    "Español (Latinoamericano)": "es-419",
    "Suomi.": "fi",
    "Français": "fr",
    "Magyar": "hu",
    "Icelandic": "is",
    "Italiano": "it",
    "ქართული": "ka",
    "қазақша": "kk",
    "Lëtzebuergesch": "lb",
    "नेपाली": "ne",
    "Nederlands": "nl",
    "Norsk": "nb",
    "Polski": "pl",
    "Português (Brasil)": "pt-br",
    "Português (Portugal)": "pt-pt",
    "Română": "ro",
    "Русский": "ru",
    "Српски": "sr",
    "Svenska": "sv",
    "Kiswahili": "sw",
    "Türkçe": "tr",
    "украї́нська": "uk",
    "Tiếng Việt": "vi",
    "简体中文": "zh"
}

def _get_language(code):
    return languages[code]

final_language = _get_language(language)
#@markdown ### Elige un nombre para tu modelo:
model_name = "Test" #@param {type:"string"}
#@markdown ---
# output:
#@markdown ###Elige la carpeta de trabajo: (se recomienda guardar en Drive)

#@markdown La carpeta de trabajo se utilizará en el preprocesamiento, pero también en el entrenamiento del modelo.
output_path = "/content/drive/MyDrive/colab/piper" #@param {type:"string"}
output_dir = output_path+"/"+model_name
if not os.path.exists(output_dir):
  os.makedirs(output_dir)
#@markdown ---
#@markdown ### Elige el formato del dataset:
dataset_format = "ljspeech" #@param ["ljspeech", "mycroft"]
#@markdown ---
#@markdown ### ¿Se trata de un conjunto de datos de un solo hablante? Si no es así, desmarca la casilla:
single_speaker = True #@param {type:"boolean"}
if single_speaker:
  force_sp = " --single-speaker"
else:
  force_sp = ""
#@markdown ---
#@markdown ###Seleccione la frecuencia de muestreo del dataset:
sample_rate = "22050" #@param ["16000", "22050"]
#@markdown ---
!mkdir /content/audio_cache
%cd /content/piper/src/python
#@markdown ###¿Quieres entrenar utilizando esta frecuencia de muestreo, pero tus audios no la tienen?
#@markdown ¡El remuestreador te ayuda a hacerlo rápidamente!
resample = False #@param {type:"boolean"}
if resample:
  !python resample.py --input_dir "/content/dataset/wavs" --output_dir "/content/dataset/wavs_resampled" --output_sr {sample_rate} --file_ext "wav"
  !mv /content/dataset/wavs_resampled/* /content/dataset/wavs
#@markdown ---
if use_whisper:
    print("El archivo de transcripción no se ha cargado. Transcribiendo estos audios usando Whisper...")
    make_dataset("/content/dataset/wavs", final_language[:2])
    print("¡Transcripción realizada! Pre-procesando...")
!python -m piper_train.preprocess \
  --language {final_language} \
  --input-dir /content/dataset \
  --cache-dir "/content/audio_cache" \
  --output-dir "{output_dir}" \
  --dataset-name "{model_name}" \
  --dataset-format {dataset_format} \
  --sample-rate {sample_rate} \
  {force_sp}

In [ ]:
#@markdown # <font color="ffc800"> **4. Ajustes.** 🧰
#@markdown ---
import json
import ipywidgets as widgets
from IPython.display import display
from google.colab import output
import os
#@markdown ### <font color="orange">**Seleccione la acción para entrenar este conjunto de datos: (LEER ANTENTAMENTE.)**

#@markdown * La opción de <font color="orange">continuar un entrenamiento</font> se explica por sí misma. Si has entrenado previamente un modelo con colab gratuito, se te ha acabado el tiempo y estás considerando entrenarlo un poco más, esto es ideal para ti. Sólo tienes que establecer los mismos ajustes que estableciste cuando entrenaste este modelo por primera vez.
#@markdown * La opción para <font color="orange">convertir un modelo de un solo hablante en un modelo multihablante</font> se explica por sí misma, y para ello es importante que hayas procesado un conjunto de datos que contenga texto y audio de todos los posibles hablantes que quieras entrenar en tu modelo.
#@markdown * La opción <font color="orange">finetune</font> se utiliza para entrenar un conjunto de datos utilizando un modelo preentrenado, es decir, entrenar sobre esos datos. <font color="orange">Esta opción es necesaria para entrenar cualquier voz, ya tengas un dataset pequeño o amplio. *(Se recomiendan más de cinco minutos de datos.)*</font>
#@markdown * La opción <font color="orange">entrenar desde cero</font> se usa para hacer modelos base, o sea, construye características como el diccionario y la forma del habla desde cero, y esto puede tardar más en converger. Para ello, se recomiendan horas de audio *(8 como mínimo)* que tengan una gran colección de fonemas.
action = "Finetune." #@param ["Continuar entrenando.", "Convertir modelo de un solo hablante a un modelo de multi hablante.", "Finetune.", "Entrenar desde cero."]
#@markdown ---
if action == "Continuar entrenando.":
    if os.path.exists(f"{output_dir}/lightning_logs/version_0/checkpoints/last.ckpt"):
        ft_command = f'--resume_from_checkpoint "{output_dir}/lightning_logs/version_0/checkpoints/last.ckpt" '
        print(f"\033[93mContinuar entrenamiento de {model_name} desde: {output_dir}/lightning_logs/version_0/checkpoints/last.ckpt")
    else:
        raise Exception("El entrenamiento no puede continuar ya que no hay ningún punto de control en el que continuar.")
elif action == "Finetune.":
    if os.path.exists(f"{output_dir}/lightning_logs/version_0/checkpoints/last.ckpt"):
        raise Exception("¡Oh no! Ya has entrenado este modelo anteriormente, no puedes elegir esta opción ya que tu progreso se perderá, y entonces tu tiempo anterior no contará. Por favor, selecciona la opción para continuar un entrenamiento.")
    else:
        ft_command = '--resume_from_checkpoint "/content/pretrained.ckpt" '
elif action == "Convertir modelo de un solo hablante a un modelo de multi hablante.":
    if not single_speaker:
        ft_command = '--resume_from_single_speaker_checkpoint "/content/pretrained.ckpt" '
    else:
        raise Exception("Este conjunto de datos no es multihablante.")
else:
    ft_command = ""
if action== "Convertir modelo de un solo hablante a un modelo de multi hablante." or action == "Finetune.":
    try:
        with open('/content/piper/notebooks/pretrained_models.json') as f:
            pretrained_models = json.load(f)
        if final_language in pretrained_models:
            models = pretrained_models[final_language]
            model_options = [(model_name, model_name) for model_name, model_url in models.items()]
            model_dropdown = widgets.Dropdown(description = "Seleccionar modelo preinstalado.", options=model_options)
            download_button = widgets.Button(description="Descargar")
            def download_model(btn):
                model_name = model_dropdown.value
                model_url = pretrained_models[final_language][model_name]
                print("\033[93mDescargando modelo preentrenado.")
                if model_url.startswith("1"):
                    !gdown -q "{model_url}" -O "/content/pretrained.ckpt"
                elif model_url.startswith("https://drive.google.com/file/d/"):
                    !gdown -q "{model_url}" -O "/content/pretrained.ckpt" --fuzzy
                else:
                    !wget -q "{model_url}" -O "/content/pretrained.ckpt"
                model_dropdown.close()
                download_button.close()
                output.clear()
                if os.path.exists("/content/pretrained.ckpt"):
                    print("\033[93m¡Modelo descargado!")
                else:
                    raise Exception("No se pudo descargar el modelo preentrenado.")
            download_button.on_click(download_model)
            display(model_dropdown, download_button)
        else:
            raise Exception(f"No hay modelos preentrenados disponibles para el idioma {final_language}")
    except FileNotFoundError:
        raise Exception("No se ha encontrado el archivo pretrained_models.json.")
else:
    print("\033[93mAdvertencia: este modelo será entrenado desde cero. Necesitas al menos 8 horas de datos para que todo funcione decentemente. Mucha suerte.")
#@markdown ### Elige el tamaño del lote basándose en este conjunto de datos:
batch_size = 12 #@param {type:"integer"}
#@markdown ---

#@markdown ### Elige la calidad para este modelo:

#@markdown * x-low - 16Khz audio, 5-7M params
#@markdown * medium - 22.05Khz audio, 15-20 params
#@markdown * high - 22.05Khz audio, 28-32M params
quality = "medium" #@param ["high", "x-low", "medium"]
#@markdown ---
#@markdown ### ¿Cada cuántas épocas quieres autoguardar los puntos de control de entrenamiento?
#@markdown Cuanto mayor sea tu conjunto de datos, debes establecer este intervalo de guardado en un valor menor, ya que las épocas pueden progresar durante más tiempo.
checkpoint_epochs = 5 #@param {type:"integer"}
#@markdown ---
#@markdown ### Intervalo para guardar los k mejores modelos:
#@markdown Póngalo a 0 si desea desactivar el guardado de varios modelos. Si este es el caso, marque la casilla de abajo. Si se establece en 1, los modelos se guardarán con el nombre de archivo epoch=xx-step=xx.ckpt, por lo que tendrá que vaciar la papelera de Drive cada cierto tiempo.
num_ckpt = 0 #@param {type: "integer"}
#@markdown ---
#@markdown ### <font color="orange">**Guardar el último modelo:**
#@markdown <font color="orange">Esta casilla debe estar marcada si deseas guardar un único modelo (last.ckpt). Guardar un único modelo sólo se aplica si num_ckpt es igual a 0. Si es así, el parámetro intervalo de pasos para autoguardar se ignora, ya que se guarda el último modelo por época; además, no tendrás que preocuparte por el almacenamiento. Siendo igual a 1, se guardará último.ckpt, pero otro modelo (model_vVersion.ckpt, este último tiene en cuenta el intervalo de épocas que establezcas), por lo que tendrías que vaciar la papelera a menudo.
#@markdown <font color="orange">No es recomendable usar esta opción en datasets ***extremadamente pequeños***, ya que al guardar el último modelo cada época, este proceso será muy rápido y el entrenador no podrá guardar el modelo completo, lo que resultaría en un last.ckpt. ***corrupto***
save_last = True # @param {type: "boolean"}
#@markdown ---
#@markdown ### Intervalo de pasos para generar muestras de audio del modelo:
log_every_n_steps = 1000 #@param {type:"integer"}
#@markdown ---
#@markdown ### Número de épocas para el entrenamiento.
max_epochs = 10000 #@param {type:"integer"}
#@markdown ---


In [ ]:
#@markdown # <font color="ffc800"> **5. Ejecuta la extensión TensorBoard.** 📈
#@markdown ---
#@markdown #### El TensorBoard se utiliza para visualizar los resultados del modelo mientras está siendo entrenado como el audio y las pérdidas.
%load_ext tensorboard
%tensorboard --logdir {output_dir}

In [ ]:
#@markdown # <font color="ffc800"> **6. Entrenar.** 🏋️‍♂️
#@markdown ---
#@markdown #### Ejecuta esta celda para entrenar tu modelo.

#@markdown ---

#@markdown ### <font color="orange">**¿Desactivar la validación?**
#@markdown Desmarcando esta casilla, permitirás entrenar el dataset completo, sin utilizar ningún archivo de audio o ejemplo como conjunto de validación. Por lo tanto, no será capaz de generar audios en el tensorboard mientras se está entrenando. Se recomienda desactivar la validación en tu dataset si es ***extremadamente pequeños.***
validation = True #@param {type:"boolean"}
if validation:
    validation_split = 0.01
    num_test_examples = 1
else:
    validation_split = 0
    num_test_examples = 0
if not save_last:
    save_last_command = ""
else:
    save_last_command = "--save_last True "
get_ipython().system(f'''
python -m piper_train \
--dataset-dir "{output_dir}" \
--accelerator 'gpu' \
--devices 1 \
--batch-size {batch_size} \
--validation-split {validation_split} \
--num-test-examples {num_test_examples} \
--quality {quality} \
--checkpoint-epochs {checkpoint_epochs} \
--num_ckpt {num_ckpt} \
{save_last_command}\
--log_every_n_steps {log_every_n_steps} \
--max_epochs {max_epochs} \
{ft_command}\
--precision 32
''')

# <font color="orange"> **¿Has terminado el entrenamiento y quieres probar el modelo?**

* ¡Si quieres ejecutar este modelo en cualquier software que Piper integre o en la misma app de Piper, exporta tu modelo usando el [cuaderno exportador de modelos](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_exportador_modelos_espa%C3%B1ol.ipynb)!
* Si quieres probar este modelo ahora mismo antes de exportarlo al formato soportado por Piper.<br>¡Prueba tu last.ckpt generado con [este cuaderno](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_inferencia_espa%C3%B1ol(ckpt).ipynb)!